# GNN

In [4]:
# Some initial setup
import os
import json
import argparse
import time
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

import matplotlib
import matplotlib.pyplot as plt

import networkx as nx
from sklearn.utils.class_weight import compute_class_weight

from tensorboardX import SummaryWriter
from fastprogress import master_bar, progress_bar

# Remove warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.sparse import SparseEfficiencyWarning
warnings.simplefilter('ignore', SparseEfficiencyWarning)

from graph_convnet.config import *
from graph_convnet.utils.graph_utils import *
from graph_convnet.utils.google_tsp_reader import GoogleTSPReader
from graph_convnet.utils.plot_utils import *
from graph_convnet.models.gcn_model import ResidualGatedGCNModel
from graph_convnet.utils.model_utils import *

In [5]:
notebook_mode = True
viz_mode = False

if notebook_mode == True:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('png')

/tmp/ipykernel_52483/2361144546.py:9: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png')


### Load TSP20 Config Files

In [6]:
config_path = "graph_convnet/configs/tsp20.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))

FileNotFoundError: [Errno 2] No such file or directory: 'graph_convnet/configs/default.json'

### Setup GPU and CUDA

In [8]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)

if torch.cuda.is_available():
    print("CUDA available, using GPU ID {}".format(1))
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed(1)
else:
    print("CUDA not available")
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    torch.manual_seed(1)

CUDA not available


### Setup Network and Parameters

In [9]:
net = nn.DataParallel(
    ResidualGatedGCNModel(config, dtypeFloat, dtypeLong)
)
if torch.cuda.is_available():
    net.cuda()

# Number of network parameters
nb_param = 0
for param in net.parameters():
    nb_param += np.prod(list(param.data.size()))
print(f"Number of parameters: {nb_param}")

# Optimizer
learning_Rate = config.learning_rate
optimizer = torch.optim.Adam(net.parameters(), lr=learning_Rate)
print(optimizer)

NameError: name 'config' is not defined

# Load previous checkpoint

In [10]:
log_dir = f"graph_convnet/logs/{config.expt_name}/"
if torch.cuda.is_available():
    checkpoint = torch.load(log_dir + "best_val_checkpoint.tar")
else:
    checkpoint = torch.load(log_dir + "best_val_checkpoint.tar", map_location="cpu")

net.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
train_loss = checkpoint['train_loss']
val_loss = checkpoint['val_loss']

for param_group in optimizer.param_groups:
    learning_Rate = param_group['lr']

print(f"Loaded checkpoint from epoch {epoch}")

NameError: name 'config' is not defined

In [11]:
def metrics_to_str(epoch, time, learning_rate, loss, err_edges, err_tour, err_tsp, pred_tour_len, gt_tour_len):
    result = ( 'epoch:{epoch:0>2d}\t'
               'time:{time:.1f}h\t'
               'lr:{learning_rate:.2e}\t'
               'loss:{loss:.4f}\t'
               # 'err_edges:{err_edges:.2f}\t'
               # 'err_tour:{err_tour:.2f}\t'
               # 'err_tsp:{err_tsp:.2f}\t'
               'pred_tour_len:{pred_tour_len:.3f}\t'
               'gt_tour_len:{gt_tour_len:.3f}'.format(
                   epoch=epoch,
                   time=time/3600,
                   learning_rate=learning_rate,
                   loss=loss,
                   # err_edges=err_edges,
                   # err_tour=err_tour,
                   # err_tsp=err_tsp,
                   pred_tour_len=pred_tour_len,
                   gt_tour_len=gt_tour_len))
    return result

### Run on test data